# RunModel with Heterogeneous Input: Python model execution

The RunModel class is capable of passing input in different formats into a single computational model. This means that the samples passed into a model can be passed as:
- floating point values
- numpy arrays
- lists 
- tuples
- lists of other iterables
- numpy arrays of other iterables
- or any combination of the above

In the examples below, we demonstrate the use of a Python computational model with inputs that are combinations of the above.

Some notes on their use:
1. UQpy converts all sample input to a numpy array with at least two dimensions. The first dimension, i.e. len(samples) must correspond to the number of samples being passed for model execution. The second dimension, i.e. len(samples[0]) must correspond to the number of variables that each sample possesses.
2. Each individual sample, i.e. sample[j], may be composed of multiple data types -- with each variable having a different data type. For example, sample[j][k] may be a floating point value and sample[j][l] may be an array of arbitrary dimension.
3. If a specific variable has multiple dimensions, the user may specify the index to be return in the input file. For example, the place holder for a variable x1 corresponding to sample[j][l] that is an array of shape (1,4) can be read as <x1[0, 3]>, which will return the final (0,3) component of samples[j][l].
4. If the user does not specify the index for a multidimensional variable, then the entire multidimensional variable is flattened and written with comma delimiters.

Michael D. Shields  
29 April 2020

### Python Model Summary
Examples 1-2:
The provided Matlab models take the sum of three random variables:  
$s = \sum_{i=1}^3 x_i$  
$x_i \sim N(0,1)$

Example 3:
The provided Matlab model takes the product of a random variable and the determinant of a random matrix:  
$z = x \det(Y)$  
$x \sim N(0,1)$  
$y$ is a 3x3 matrix of standard normal random variables.

The Python model may be provided as either a class or a function. The examples below explore both cases.

In [1]:
from UQpy.SampleMethods import MCS
from UQpy.RunModel import RunModel
import matplotlib.pyplot as plt
import time
import numpy as np

### Pick which model to run
Options:
- 'all'
- 'scalar'
- 'vector'
- 'mixed'
- 'fixed'

In [2]:
pick_model = 'all'

## Example 1: Three scalar random variables
In this example, we pass three scalar random variables. Note that this is different from assigning a single variable with three components, which will be handled in the following example. 

Here we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

In [3]:
if pick_model == 'scalar' or pick_model =='vector' or pick_model == 'all':
    # Call MCS to generate samples
    # THIS WILL NEED TO BE REWRITTEN WITH DISTRIBUTION AND MCS UPDATES --------------------------------------------
    x_mcs = MCS(dist_name=['Normal','Normal','Normal'], dist_params=[[0,1],[0,1],[0,1]], nsamples=5, 
                var_names = ['var1', 'var11', 'var111']) 
    # -------------------------------------------------------------------------------------------------------------

    # UQpy returns samples as an ndarray. Convert them to a list for part 1.2
    x_mcs_list = list(x_mcs.samples)
    print("Monte Carlo samples of three random variables from a standard normal distribution.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs.samples))
    print('Number of samples:', len(x_mcs.samples))
    print('Dimensions of samples:', np.shape(x_mcs.samples))
    print('Samples')
    print(x_mcs.samples)
    print()
    print('Samples stored as a list:')
    print('Data type:', type(x_mcs_list))   
    print('Number of samples:', len(x_mcs_list))
    print('Dimensions of samples:', np.shape(x_mcs_list))
    print('Samples:')
    print(x_mcs_list)

Monte Carlo samples of three random variables from a standard normal distribution.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 3)
Samples
[[-0.74872205  1.27075369  0.25163963]
 [ 0.14681843  0.88266692  0.97283151]
 [-0.09694895 -0.4911442  -1.7552827 ]
 [ 0.24977597  0.09712931 -1.08294372]
 [-0.46119419 -1.50942848  0.64172376]]

Samples stored as a list:
Data type: <class 'list'>
Number of samples: 5
Dimensions of samples: (5, 3)
Samples:
[array([-0.74872205,  1.27075369,  0.25163963]), array([0.14681843, 0.88266692, 0.97283151]), array([-0.09694895, -0.4911442 , -1.7552827 ]), array([ 0.24977597,  0.09712931, -1.08294372]), array([-0.46119419, -1.50942848,  0.64172376])]


### 1.1 Pass samples as ndarray, Python class called, serial execution  
This examples uses the following files:
- model_script = python_model.py

In [4]:
if pick_model == 'scalar' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m11 = RunModel(ntasks=1, model_script='python_model.py', model_object_name='SumRVs', model_dir='Python_Runs')
    m11.run(samples=x_mcs.samples,)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m11.qoi_list)


Time for serial execution:
0.012399911880493164

The values returned from the Matlab simulation:
[0.7736712678997194, 2.0023168726271288, -2.3433758468404147, -0.7360384447587722, -1.3288989109053513]


### 1.2 Pass samples as list, Python function called, parallel execution  
This examples uses the following files:
- model_script = python_model.py

In [5]:
if pick_model == 'scalar' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m12 = RunModel(samples=x_mcs_list, ntasks=2, model_script='python_model.py', 
                 model_object_name='sum_rvs', model_dir='Python_Runs')
    t_par_python = time.time() - t
    print("\nTime for parallel execution:")
    print(t_par_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m12.qoi_list)


Time for parallel execution:
0.03223586082458496

The values returned from the Matlab simulation:
[array([0.77367127]), array([2.00231687]), array([-2.34337585]), array([-0.73603844]), array([-1.32889891])]


## Example 2: Single tri-variate random variable
In this example, we pass three random variables in as a trivariate random variable. Note that this is different from assigning three scalar random variables, which was be handled in Example 1.

Again, we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

### Restructure the samples
To pass the samples in as a single tri-variate variable, we need reshape the samples from shape (5, 3) to shape (5, 1, 3)

In [6]:
if pick_model == 'vector' or pick_model == 'all':
    x_mcs_tri = x_mcs.samples.reshape(5, 1, 3)
    x_mcs_tri_list = list(x_mcs_tri)

    print("Monte Carlo samples of three random variables from a standard normal distribution.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs_tri))
    print('Number of samples:', len(x_mcs_tri))
    print('Dimensions of samples:', np.shape(x_mcs_tri))
    print('Samples')
    print(x_mcs_tri)
    print()
    print('Samples stored as a list:')
    print('Data type:', type(x_mcs_tri_list))   
    print('Number of samples:', len(x_mcs_tri_list))
    print('Dimensions of samples:', np.shape(x_mcs_tri_list))
    print('Samples:')
    print(x_mcs_tri_list)

Monte Carlo samples of three random variables from a standard normal distribution.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 1, 3)
Samples
[[[-0.74872205  1.27075369  0.25163963]]

 [[ 0.14681843  0.88266692  0.97283151]]

 [[-0.09694895 -0.4911442  -1.7552827 ]]

 [[ 0.24977597  0.09712931 -1.08294372]]

 [[-0.46119419 -1.50942848  0.64172376]]]

Samples stored as a list:
Data type: <class 'list'>
Number of samples: 5
Dimensions of samples: (5, 1, 3)
Samples:
[array([[-0.74872205,  1.27075369,  0.25163963]]), array([[0.14681843, 0.88266692, 0.97283151]]), array([[-0.09694895, -0.4911442 , -1.7552827 ]]), array([[ 0.24977597,  0.09712931, -1.08294372]]), array([[-0.46119419, -1.50942848,  0.64172376]])]


### 2.1 Pass samples as ndarray, Python function called, serial execution
This example uses the following files:
- model_script = python_model.py

In [7]:
if pick_model == 'vector' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m21 = RunModel(samples=x_mcs_tri, ntasks=1, model_script='python_model.py', 
                 model_object_name='sum_rvs_vec', model_dir='Python_Runs')
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m21.qoi_list)


Time for serial execution:
0.01606297492980957

The values returned from the Matlab simulation:
[array([0.77367127]), array([2.00231687]), array([-2.34337585]), array([-0.73603844]), array([-1.32889891])]


### 2.2 Pass samples as list, Python class called, parallel execution
This example uses the following files:
- model_script = python_model.py

In [8]:
if pick_model == 'vector' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m22 = RunModel(samples=x_mcs_tri_list, ntasks=2, model_script='python_model.py', 
                 model_object_name='SumRVs', model_dir='Python_Runs')
    t_par_python = time.time() - t
    print("\nTime for parallel execution:")
    print(t_par_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m22.qoi_list)


Time for parallel execution:
0.03136897087097168

The values returned from the Matlab simulation:
[array([0.77367127]), array([2.00231687]), array([-2.34337585]), array([-0.73603844]), array([-1.32889891])]


## Example 3: Passing a scalar and an array to RunModel
In this example, we pass a single scalar random variable as well as an array into a Matlab model.

Again, we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

In [9]:
if pick_model == 'mixed' or pick_model =='vector' or pick_model == 'all':
    # Call MCS to generate samples
    # THIS WILL NEED TO BE REWRITTEN WITH DISTRIBUTION AND MCS UPDATES --------------------------------------------
    # First generate the scalar random variable
    x_mcs1 = MCS(dist_name=['Normal'], dist_params=[[0,1]], nsamples=5, var_names = ['var1']) 
    # Next generate a 3x3 random matrix
    x_mcs2 = MCS(dist_name=['Normal','Normal','Normal'], dist_params=[[0,1],[0,1],[0,1]], nsamples=15)
    x_mcs_array = x_mcs2.samples.reshape((5,3,3))
    # -------------------------------------------------------------------------------------------------------------
    
    print("Monte Carlo samples of a single random variable from a standard normal distribution.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs1.samples))
    print('Number of samples:', len(x_mcs1.samples))
    print('Dimensions of samples:', np.shape(x_mcs1.samples))
    print('Samples')
    print(x_mcs1.samples)
    print()
    print("Monte Carlo samples of a 3x3 matrix of standard normal random variables.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs_array))
    print('Number of samples:', len(x_mcs_array))
    print('Dimensions of samples:', np.shape(x_mcs_array))
    print('Samples')
    print(x_mcs_array)
    print()
    
    # Create a set of samples to be passed into RunModel
    # Here we need to create the mixed samples such that each sample has a single scalar and a single 3x3 matrix.
    # This data structure is essential to passing the input to UQpy correctly.
    x_mixed = []
    for i in range(5):
        x_mixed.append([x_mcs1.samples[i], x_mcs_array[i]])
        
    print("Combined samples with a scalar and a 3x3 matrix of standard normal random variables.")
    print('Samples stored as a list:')
    print('Data type:', type(x_mixed))
    print('Number of samples:', len(x_mixed))
    print('Dimensions of samples:', np.shape(x_mixed))
    print('Samples')
    print(x_mixed)
    print()
    
    x_mixed_array = np.atleast_2d(np.asarray(x_mixed))
    print("Combined samples with a scalar and a 3x3 matrix of standard normal random variables.")
    print('Samples stored as ndarray:')
    print('Data type:', type(x_mixed_array))
    print('Number of samples:', len(x_mixed_array))
    print('Dimensions of samples:', np.shape(x_mixed_array))
    print('Samples')
    print(x_mixed_array)
    print()
    
    # Notice that, in both the ndarray case and the list case, the samples have dimension (5,2). That is, there
    # are five samples of two variables. The first variable is a scalar. The second variable is a 3x3 matrix.

Monte Carlo samples of a single random variable from a standard normal distribution.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 1)
Samples
[[-0.45855936]
 [ 0.36754108]
 [ 1.44041493]
 [ 0.7546697 ]
 [ 1.7476449 ]]

Monte Carlo samples of a 3x3 matrix of standard normal random variables.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 3, 3)
Samples
[[[ 0.55753922  2.34328819 -1.00092479]
  [-0.43379101  1.84753919 -1.2077839 ]
  [ 1.04777473 -1.67862715 -1.38296314]]

 [[-0.55290311  1.03915508 -0.02892851]
  [ 1.75566362 -0.07591685 -1.23532293]
  [-0.46088052  0.67969104  2.22905913]]

 [[-1.77331278 -0.14651258 -1.28718587]
  [ 1.49193257  1.64814527 -0.226943  ]
  [ 1.544142    1.07544422  0.42373174]]

 [[-1.15185765 -0.27227344  0.05375925]
  [-0.59408261  1.36637931 -0.79731448]
  [ 0.04376446  0.3029877  -1.09950494]]

 [[-2.13010581  0.36398522  0.

### 3.1 Pass samples as ndarray, Python class called, serial execution 
This examples uses the following files:
- model_script = python_model.py

In [10]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m31 = RunModel(samples=x_mixed_array, ntasks=1, model_script='python_model.py', 
                 model_object_name='DetRVs', model_dir='Python_Runs', vec=False)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m31.qoi_list)


Time for serial execution:
0.0217740535736084

The values returned from the Matlab simulation:
[array([2.62174399]), array([-1.42578919]), array([-0.456174]), array([1.22760495]), array([4.26632608])]


### 3.2 Pass samples as list, Python function called, parallel execution 
This examples uses the following files:
- model_script = python_model.py

In [11]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    # Note that the parallel model_object handles only one sample at a time.
    t = time.time()
    m32 = RunModel(samples=x_mixed, ntasks=1, model_script='python_model.py', 
                 model_object_name='det_rvs_par', model_dir='Python_Runs', vec=False)
    t_par_python = time.time() - t
    print("\nTime for parallel execution:")
    print(t_par_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m32.qoi_list)


Time for parallel execution:
0.011741161346435547

The values returned from the Matlab simulation:
[array([2.62174399]), array([-1.42578919]), array([-0.456174]), array([1.22760495]), array([4.26632608])]


## Example 4: Passing a fixed variable and an array of Random Variables to RunModel
In this example, we pass a fixed value coefficient as well as an array of random variables into a Python model.

Again, we will pass the samples both as an ndarray and as a list. Recall that UQpy converts all samples into an ndarray of at least two dimensions internally.

In [12]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call MCS to generate samples
    x = 2.5
    
    print('Constant Coefficient:')
    print(x)
    print()
    
    print("Monte Carlo samples of a 3x3 matrix of standard normal random variables.")
    print('Samples stored as an array:')
    print('Data type:', type(x_mcs_array))
    print('Number of samples:', len(x_mcs_array))
    print('Dimensions of samples:', np.shape(x_mcs_array))
    print('Samples')
    print(x_mcs_array)
    print()
    
    x_mcs_list = list(x_mcs_array)

    print("3x3 matrix of standard normal random variables.")
    print('Samples stored as ndarray:')
    print('Data type:', type(x_mcs_list))
    print('Number of samples:', len(x_mcs_list))
    print('Dimensions of samples:', np.shape(x_mcs_list))
    print('Samples')
    print(x_mcs_list)
    print()
    
    # Notice that, in both the ndarray case and the list case, the samples have dimension (5,2). That is, there
    # are five samples of two variables. The first variable is a scalar. The second variable is a 3x3 matrix.

Constant Coefficient:
2.5

Monte Carlo samples of a 3x3 matrix of standard normal random variables.
Samples stored as an array:
Data type: <class 'numpy.ndarray'>
Number of samples: 5
Dimensions of samples: (5, 3, 3)
Samples
[[[ 0.55753922  2.34328819 -1.00092479]
  [-0.43379101  1.84753919 -1.2077839 ]
  [ 1.04777473 -1.67862715 -1.38296314]]

 [[-0.55290311  1.03915508 -0.02892851]
  [ 1.75566362 -0.07591685 -1.23532293]
  [-0.46088052  0.67969104  2.22905913]]

 [[-1.77331278 -0.14651258 -1.28718587]
  [ 1.49193257  1.64814527 -0.226943  ]
  [ 1.544142    1.07544422  0.42373174]]

 [[-1.15185765 -0.27227344  0.05375925]
  [-0.59408261  1.36637931 -0.79731448]
  [ 0.04376446  0.3029877  -1.09950494]]

 [[-2.13010581  0.36398522  0.34588439]
  [-1.35521648  0.87214484  0.47646223]
  [ 1.17198855 -0.50744154 -2.1023544 ]]]

3x3 matrix of standard normal random variables.
Samples stored as ndarray:
Data type: <class 'list'>
Number of samples: 5
Dimensions of samples: (5, 3, 3)
Samples
[

### 4.1 Pass samples as ndarray, Python class called, serial execution 
This examples uses the following files:
- model_script = python_model.py

In [13]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m41 = RunModel(samples=x_mcs_array, ntasks=1, model_script='python_model.py', 
                 model_object_name='det_rvs_fixed', model_dir='Python_Runs', vec=False, coeff=x)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m41.qoi_list)


Time for serial execution:
0.013948917388916016

The values returned from the Matlab simulation:
[-14.29337298443281, -9.698161956313964, -0.7917406072915002, 4.066696182412885, 6.102964755267562]


### 4.2 Pass samples as list, Python class called, serial execution 
This examples uses the following files:
- model_script = python_model.py

In [14]:
if pick_model == 'mixed' or pick_model == 'all':
    # Call to RunModel - Here we run the model while instantiating the RunModel object. 
    t = time.time()
    m42 = RunModel(samples=x_mcs_list, ntasks=1, model_script='python_model.py', 
                 model_object_name='det_rvs_fixed', model_dir='Python_Runs', vec=False, coeff=x)
    t_ser_python = time.time() - t
    print("\nTime for serial execution:")
    print(t_ser_python)
    print()
    print("The values returned from the Matlab simulation:")
    print(m42.qoi_list)


Time for serial execution:
0.011161088943481445

The values returned from the Matlab simulation:
[-14.29337298443281, -9.698161956313964, -0.7917406072915002, 4.066696182412885, 6.102964755267562]
